In [ ]:
"""
##### Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

We have a dataset of ecommerce item description. Total 4 categories,
1. Household
2. Electronics
3. Clothing and Accessories
4. Books

The task at hand is to classify a product into one of the above 4 categories based on the product description
"""

In [1]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [2]:
# Dropping NA

df.dropna(inplace=True)
df.shape

(50424, 2)

In [3]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [4]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [6]:
# When you train a fasttext model, it expects labels to be specified with __label__ prefix. 
# We will just create a third column in the dataframe that has __label__ as well as the product description

In [7]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [8]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


In [ ]:
"""
**Pre-procesing**
1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case
"""

In [9]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']',' ', text)
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [11]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [13]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [14]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [15]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

Read 4M words
Number of words:  79392
Number of labels: 4
Progress: 100.0% words/sec/thread: 2390355 lr:  0.000000 avg.loss:  0.187875 ETA:   0h 0m 0s0m 0s


(10083, 0.9670732916790638, 0.9670732916790638)

In [16]:
# First parameter (10084) is test size. Second and third parameters are precision and recall respectively. 
# You can see we are getting around 96% precision which is pretty good

In [17]:
# let's predict for few product description

model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99702644]))

In [18]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [19]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000966]))

In [20]:
model.get_nearest_neighbors("painting")

[(0.9988825917243958, 'squarical'),
 (0.9988692402839661, 'nests'),
 (0.9988675117492676, '1866'),
 (0.9988639950752258, '450f'),
 (0.9988598227500916, 'sidewalls'),
 (0.998857855796814, 'gobble'),
 (0.998857855796814, 'moneythey'),
 (0.998857855796814, 'bidthe'),
 (0.998857855796814, 'includedplease'),
 (0.998857855796814, 'banknote')]

In [21]:
model.get_nearest_neighbors("sony")

[(0.9993653893470764, 'preconfigured'),
 (0.9993622303009033, 'h9r'),
 (0.999333918094635, '3ms'),
 (0.9993264675140381, 'prismatic'),
 (0.9993264675140381, 'turboboost'),
 (0.9993264675140381, 'speedstep'),
 (0.9993264675140381, '2le'),
 (0.9993264675140381, 'trig'),
 (0.9993264675140381, 'avx2'),
 (0.999287486076355, 'upgradeable')]

In [22]:
model.get_nearest_neighbors("banglore")

[(0.0, 'of'),
 (0.0, 'a'),
 (0.0, 'for'),
 (0.0, 'in'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'adinfinitum'),
 (0.0, 'zoomplus'),
 (0.0, '1080mm'),
 (0.0, 'sx430')]